**Define all the imports**

In [1]:
import os
import cv2
import numpy as np

from torchvision import transforms as T
import torch
import torch.nn as nn

import matplotlib.pyplot as plt

import torchvision

from collections import OrderedDict
import torch.optim as optim

**Apple Dataset**

In [2]:
class Apple_Dataset(object):
    def __init__(self, root_dir, img_transform, mask_transform):
        self.root_dir = root_dir
        self.img_transform = img_transform
        self.mask_transform = mask_transform

        # Load all image and mask files, sorting them to ensure they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root_dir, "masks"))))

    def __getitem__(self, idx):
        # Load images and masks
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])
        mask_path = os.path.join(self.root_dir, "masks", self.masks[idx])

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, 0)


        # Split the color-encoded masks into a set of binary masks
        target = np.array((mask>0)*255, dtype=np.uint8)
        

        img = self.img_transform(cv2.resize(img, (512,512)) )
        target = self.mask_transform(cv2.resize(target, (512,512)))
        

        # return tensors
        return img, target

    def __len__(self):
        return len(self.imgs)

**Model Class**

In [3]:
class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

class DiceLoss(nn.Module):

    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 1.0

    def forward(self, y_pred, y_true):
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (
            y_pred.sum() + y_true.sum() + self.smooth
        )
        return 1. - dsc

**Utils**

In [4]:
class Utils(object):
    def __init__(self):
        pass
    
    def get_transform(self, ifimg):
        transforms = [T.ToTensor()]
        if ifimg:
            transforms.append(T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)))
        return T.Compose(transforms)


    def my_collate_fn(self, batch):
        images = [each[0] for each in batch]
        targets = [each[1] for each in batch]
        return images, targets


    def generate_bbox(self, img, boxes):
        for (x_min, y_min, x_max, y_max) in boxes:
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            
    def get_frcnn_model_instance(self, num_classes):
        # load an instance segmentation model pre-trained pre-trained on COCO
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

        # get number of input features for the classifier
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        # replace the pre-trained head with a new one
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        return model
    

**Training Using Unet**

In [5]:
root_path = '../input/minneapple/Dataset/Dataset'
out_folder = '/kaggle/working/apple_counting/'
num_epochs = 150

util = Utils()
train_dataset = Apple_Dataset(os.path.join(root_path, 'train'), util.get_transform(True), util.get_transform(False))
val_dataset = Apple_Dataset(os.path.join(root_path, 'val'), util.get_transform(True), util.get_transform(False))
data_loader = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=6, shuffle=True, num_workers=4),
               'val': torch.utils.data.DataLoader(val_dataset, batch_size=6, shuffle=True, num_workers=4)}

device = torch.device("cpu" if not torch.cuda.is_available() else "cuda:0")

model = UNet(in_channels=3, out_channels=1)
model.to(device)

dsc_loss = DiceLoss()

optimizer = optim.Adam(model.parameters(), lr=0.0001)


if not os.path.exists(out_folder + "/Models/"):
    os.makedirs(out_folder + "/Models/")

for epoch in range(num_epochs):
    for category in ['train', 'val']:
        loss_epoch = []
        for i, (images, targets) in enumerate(data_loader[category]):
            x = images.to(device)
            y_true = targets.to(device)

            if category == 'train':
                model.train()
                optimizer.zero_grad()

                with torch.set_grad_enabled(True):
                    y_pred = model(x)
                    loss = dsc_loss(y_pred, y_true)


                loss.backward()
                optimizer.step()
            else:
                model.eval()
                y_pred = model(x)
                loss = dsc_loss(y_pred, y_true)

            loss_epoch.append(loss.item())
        
        print ("{} loss at epoch {} of {}: {}".format(category, epoch, num_epochs, sum(loss_epoch)/len(loss_epoch)))
    
    if epoch % 50 == 0:
        torch.save(model.state_dict(), out_folder + "/Models/UNet_18Dec2020_{}.pt".format(epoch))

torch.save(model.state_dict(), out_folder + "/Models/UNet_18Dec2020_{}.pt".format(epoch))

train loss at epoch 0 of 150: 0.8985431914898887
val loss at epoch 0 of 150: 0.8560212124948916
train loss at epoch 1 of 150: 0.8611714822142872
val loss at epoch 1 of 150: 0.8381267578705497
train loss at epoch 2 of 150: 0.8466356298816737
val loss at epoch 2 of 150: 0.8244355554166047
train loss at epoch 3 of 150: 0.8336120301218175
val loss at epoch 3 of 150: 0.8090829564177472
train loss at epoch 4 of 150: 0.8209315449444216
val loss at epoch 4 of 150: 0.8037484728771708
train loss at epoch 5 of 150: 0.8088104920600777
val loss at epoch 5 of 150: 0.7817859520082888
train loss at epoch 6 of 150: 0.7931607837107644
val loss at epoch 6 of 150: 0.7658900799958602
train loss at epoch 7 of 150: 0.777122654131989
val loss at epoch 7 of 150: 0.7537402795708698
train loss at epoch 8 of 150: 0.7609520830325226
val loss at epoch 8 of 150: 0.7328215967053953
train loss at epoch 9 of 150: 0.7397801066512493
val loss at epoch 9 of 150: 0.7110292911529541
train loss at epoch 10 of 150: 0.72208509

**Code Test**

In [6]:
model.eval()

sample_img = '../input/minneapple/detection/test/images/dataset1_back_1.png'

img = cv2.imread(sample_img)

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


plt.figure()
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(img)

img = cv2.resize(img, (512,512))

trans = util.get_transform(True)
img = trans(img)
img = img.to(device)

y = model(img[None,:,:,:])

mask = np.array(y.cpu().detach().data[0].permute(1,2,0) *255, dtype = np.uint8)
mask = cv2.resize(mask, (720, 1280))


plt.subplot(122)
plt.imshow(mask)
plt.show()


# root_path = '../input/minneapple/detection/train'

# util = Utils()

# dataset = Apple_Dataset(root_path, util.get_transform(True), util.get_transform(False))
# data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

# for i, (images, targets) in enumerate(data_loader):
#     if i > 0:
#         break
        
#     print(images.shape, targets.shape)
#     print(targets[0])
#     print(images[0])

error: OpenCV(4.4.0) /tmp/pip-req-build-dglzv4yn/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
